# `memgraph` as tutorial from nvidia blogs

https://developer.nvidia.com/blog/running-large-scale-graph-analytics-with-memgraph-and-nvidia-cugraph-algorithms/

please note: in order to read the csv files in memgraph docker container, I have to change the permission of the .csv files to 101:101. It is not recommended but that is the only way that I've found so far.

In [ ]:
from gqlalchemy import Memgraph

memgraph = Memgraph("127.0.0.1", 7687)

memgraph.drop_database()

In [ ]:
# memgraph.execute("STORAGE MODE ON_DISK_TRANSACTIONAL;")
# memgraph.execute("STORAGE MODE IN_MEMORY_TRANSACTIONAL;")

In [ ]:
restuls = memgraph.execute_and_fetch("SHOW STORAGE INFO;")
list(restuls)

As Memgraph imports queries faster when data has indices, create them for all the nodes with the label `:Project` on the `id` property.

In [ ]:
memgraph.execute("CREATE INDEX ON :Project(id);")
memgraph.execute("CREATE INDEX ON :Lender(id);")
memgraph.execute("CREATE INDEX ON :Tag(name);")

In [ ]:
memgraph.execute(
    """
    LOAD CSV FROM "/csv_data/tags.csv" WITH HEADER AS row
    CREATE (:Tag {name: row['name:ID']});
    """
)

In [ ]:
memgraph.execute(
    """
    LOAD CSV FROM "/csv_data/lenders.csv" WITH HEADER AS row
    CREATE (:Lender {id: toInteger(row['lender_id']), name: row['lender_name'], publicId: row['lender_publicId']});
    """
)

In [ ]:
memgraph.execute(
    """
    LOAD CSV FROM "/csv_data/loans.csv" WITH HEADER AS row
    CREATE (:Project {
        id: toInteger(row['id:ID(Loan-ID)']), 
        name: row['name']
    });
    """
)

In [ ]:
memgraph.execute(
    """
    LOAD CSV FROM "/csv_data/lender_loan.csv" WITH HEADER AS row
    MATCH (loan:Project {id: toInteger(row[':END_ID(Loan-ID)'])}), (lender:Lender {id: toInteger(row[':START_ID(Lender-ID)'])})
    CREATE (lender)-[:LEND {amount: toFloat(row['shareAmount'])}]->(loan)
    """
)

In [ ]:
memgraph.execute(
    """
LOAD CSV FROM "/csv_data/loan_tags.csv" WITH HEADER AS row
MATCH (loan:Project {id: toInteger(row[':START_ID(Loan-ID)'])}), (tag:Tag {name: row[':END_ID']})
CREATE (loan)-[:TAGGED_WITH]->(tag)
"""
)

In [ ]:
memgraph.execute(
    """
MATCH (t1:Tag)<-[:TAGGED_WITH]-(loan:Project)-[:TAGGED_WITH]->(t2:Tag)
WHERE ID(t1) > ID(t2)
WITH collect(DISTINCT loan) as loans, t1, t2
MERGE (t1)-[:COMMON_LOANS {weight: size(loans)}]->(t2)
"""
)

In [ ]:
# INTEREST relationship
memgraph.execute(
    """
MATCH (lender:Lender)-[lend:LEND]->(loan:Project)-[t:TAGGED_WITH]->(tag:Tag)
MERGE (lender)-[i:INTEREST]->(tag)
ON CREATE SET i.weight = 1
ON MATCH SET i.weight = i.weight + 1
"""
)

In [ ]:
# SHARE_TAGS relationship (without weight)

# Analysis the `tag-tag` relationship

## pagerank

In [ ]:
results = memgraph.execute_and_fetch(
    """
    MATCH p=(:Tag)-[:COMMON_LOANS]->(:Tag)
    WITH project(p) AS tagtag_graph
    CALL pagerank.get(tagtag_graph) YIELD node, rank
    SET node.rank = rank
    RETURN node.name, rank
    ORDER BY rank DESC;
    """
)

for dict_result in results:
    print(f"tag: {dict_result['node.name']}, rank: {dict_result['rank']}")

visualize the pagerank, using *memgraph lab*

query:

```
MATCH (t:Tag)
WITH t
ORDER BY t.rank DESC
LIMIT 2
MATCH path=(t)<-[:COMMON_LOANS]-(m:Tag)
RETURN path;
```

Change the node size based on the rank:

```
@NodeStyle HasProperty(node, "rank") {
  size: Round(Mul(Property(node, "rank"), 100))
}
```

## `Louvain` community

In [ ]:
memgraph.execute(
    """
MATCH (t:Tag), (:Tag)-[e:COMMON_LOANS]->(:Tag)
WITH COLLECT(t) AS nodes, COLLECT(e) AS relationships
CALL community_detection.get_subgraph(nodes, relationships)
YIELD node, community_id
SET node.community_id = community_id;
    """
)

In [ ]:
results = memgraph.execute_and_fetch(
    """
MATCH (t:Tag)
WITH DISTINCT t.community_id as community_id
RETURN count(community_id ) as num_of_clusters;
    """
)
# you will get only 1 result
result = list(results)[0]

# don't forget that results are saved in a dict
print(f"Number of clusters: {result['num_of_clusters']}")

display Tags with community

```
MATCH p=(Tag)-[:COMMON_LOANS]->(:Tag)
RETURN p
```

style change


```
// Cache map to keep a selected color for each community
Define(ColorByInt, AsMap())
Define(GetColorByInt, Function(i, Coalesce(
  Get(ColorByInt, AsText(i)),
  Set(ColorByInt, AsText(i), GetNextColor())
)))

// Feel free to uncomment the lines below to set up a custom style for the specific node property
@NodeStyle HasProperty(node, "community_id") {
  Define(COLOR, GetColorByInt(Property(node, "community_id")))

  color: COLOR
  color-hover: Lighter(COLOR)
  color-selected: Darker(COLOR)
}
```